### Preparação do ambiente

In [ ]:
# Instalação de dependências
%pip install -r requirements.txt

In [1]:
import tensorflow
import pandas as pd
from typing import List, Tuple
import numpy as np
import regex
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from sklearn.model_selection import train_test_split

### Loading e pre processamento dos dados

In [2]:
# Definição de caminhos dos arquivos de corpus
train_file = "Penn Treebank/Secs0-18 - training"
dev_file   = "Penn Treebank/Secs19-21 - development"
test_file  = "Penn Treebank/Secs22-24 - testing"

In [3]:
# ------------------------------
# Funções de pré-processamento do texto
def carregar_corpus(caminho_arquivo: str) -> str:
    """
    Lê o arquivo completo em utf-8 e retorna como string.
    """
    with open(caminho_arquivo, "r", encoding="utf-8") as f:
        return f.read()

def dividir_em_sentencas(texto: str) -> List[str]:
    """
    Divide o texto em sentenças, assumindo uma sentença por linha.
    """
    return texto.strip().split("\n")

def processar_sentenca(sentenca: str) -> List[Tuple[str, str]]:
    """
    Separa tokens de formato palavra_TAG em pares (palavra, tag).
    Converte para lowercase, exceto nomes próprios (NNP, NNPS).
    """
    tokens = sentenca.strip().split()
    pares = []
    for token in tokens:
        if "_" in token:
            palavra, tag = token.rsplit("_", 1)
            if not(tag == 'NNP' or tag == 'NNPS'):
                palavra = palavra.lower()
            pares.append((palavra, tag))
    return pares

def construir_dataframe(sentencas: List[str]) -> pd.DataFrame:
    """
    Cria um DataFrame 'longo' com colunas:
    sentenca (ID), palavra, tag e posicao_na_sentenca.
    """
    dados = []
    for sent_id, sentenca in enumerate(sentencas):
        palavras_tags = processar_sentenca(sentenca)
        for posicao, (palavra, tag) in enumerate(palavras_tags):
            dados.append({
                "sentenca": sent_id + 1,
                "palavra": palavra,
                "tag": tag,
                "posicao_na_sentenca": posicao
            })

    return pd.DataFrame(dados)

In [4]:
# Carregando e processando os datasets
texto_raw_train = carregar_corpus(train_file)
texto_raw_dev = carregar_corpus(dev_file)
texto_raw_teste = carregar_corpus(test_file)
sentencas_train = dividir_em_sentencas(texto_raw_train + texto_raw_dev + texto_raw_teste)
df_treino = construir_dataframe(sentencas_train)
df_treino.fillna(method="ffill", inplace=True)

# Primeiras linhas
df_treino.head()

C:\Users\lucas\AppData\Local\Temp\ipykernel_21124\2364600724.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_treino.fillna(method="ffill", inplace=True)


,sentenca,palavra,tag,posicao_na_sentenca
0,1,Pierre,NNP,0
1,1,Vinken,NNP,1
2,1,",",",",2
3,1,61,CD,3
4,1,years,NNS,4


#### Vocabulário e lista de tags

In [5]:
vocab = df_treino["palavra"].unique().tolist()
vocab.append("<PAD>") # Padding

vocab

['Pierre',
 'Vinken',
 ',',
 '61',
 'years',
 'old',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'Nov.',
 '29',
 '.',
 'Mr.',
 'is',
 'chairman',
 'of',
 'Elsevier',
 'N.V.',
 'Dutch',
 'publishing',
 'group',
 'Rudolph',
 'Agnew',
 '55',
 'and',
 'former',
 'Consolidated',
 'Gold',
 'Fields',
 'PLC',
 'was',
 'named',
 'this',
 'british',
 'industrial',
 'conglomerate',
 'form',
 'asbestos',
 'once',
 'used',
 'to',
 'make',
 'Kent',
 'cigarette',
 'filters',
 'has',
 'caused',
 'high',
 'percentage',
 'cancer',
 'deaths',
 'among',
 'workers',
 'exposed',
 'it',
 'more',
 'than',
 '30',
 'ago',
 'researchers',
 'reported',
 'fiber',
 'crocidolite',
 'unusually',
 'resilient',
 'enters',
 'lungs',
 'with',
 'even',
 'brief',
 'exposures',
 'causing',
 'symptoms',
 'that',
 'show',
 'up',
 'decades',
 'later',
 'said',
 'Lorillard',
 'Inc.',
 'unit',
 'new',
 'york-based',
 'Loews',
 'Corp.',
 'makes',
 'cigarettes',
 'stopped',
 'using',
 'in',
 'it

In [6]:
tags = df_treino["tag"].unique().tolist()

tags

['NNP',
 ',',
 'CD',
 'NNS',
 'JJ',
 'MD',
 'VB',
 'DT',
 'NN',
 'IN',
 '.',
 'VBZ',
 'VBG',
 'CC',
 'VBD',
 'VBN',
 'RB',
 'TO',
 'PRP',
 'RBR',
 'WDT',
 'VBP',
 'RP',
 'PRP$',
 'JJS',
 'POS',
 '``',
 'EX',
 "''",
 'WP',
 ':',
 'JJR',
 'WRB',
 '$',
 'NNPS',
 'WP$',
 '-LRB-',
 '-RRB-',
 'PDT',
 'RBS',
 'FW',
 'UH',
 'SYM',
 'LS',
 '#']

#### Separar em sentenças

In [7]:
def ler_sentencas(dados):
    agg = lambda sentenca: [(w, p) for w, p in zip(sentenca["palavra"].values.tolist(), sentenca["tag"].values.tolist())]
    agrupado = dados.groupby("sentenca").apply(agg)
    sentencas = [sentenca for sentenca in agrupado]
    return sentencas

In [8]:
sentencas_treino = ler_sentencas(df_treino)

C:\Users\lucas\AppData\Local\Temp\ipykernel_21124\1560216814.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agrupado = dados.groupby("sentenca").apply(agg)


In [9]:
sentencas_treino[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

### Geração de embeddings

In [10]:
# Convertendo palavras e tags em números
vocab2id = {w: i for i, w in enumerate(vocab)}
tag2id = {t: i for i, t in enumerate(tags)}

In [11]:
vocab2id

{'Pierre': 0,
 'Vinken': 1,
 ',': 2,
 '61': 3,
 'years': 4,
 'old': 5,
 'will': 6,
 'join': 7,
 'the': 8,
 'board': 9,
 'as': 10,
 'a': 11,
 'nonexecutive': 12,
 'director': 13,
 'Nov.': 14,
 '29': 15,
 '.': 16,
 'Mr.': 17,
 'is': 18,
 'chairman': 19,
 'of': 20,
 'Elsevier': 21,
 'N.V.': 22,
 'Dutch': 23,
 'publishing': 24,
 'group': 25,
 'Rudolph': 26,
 'Agnew': 27,
 '55': 28,
 'and': 29,
 'former': 30,
 'Consolidated': 31,
 'Gold': 32,
 'Fields': 33,
 'PLC': 34,
 'was': 35,
 'named': 36,
 'this': 37,
 'british': 38,
 'industrial': 39,
 'conglomerate': 40,
 'form': 41,
 'asbestos': 42,
 'once': 43,
 'used': 44,
 'to': 45,
 'make': 46,
 'Kent': 47,
 'cigarette': 48,
 'filters': 49,
 'has': 50,
 'caused': 51,
 'high': 52,
 'percentage': 53,
 'cancer': 54,
 'deaths': 55,
 'among': 56,
 'workers': 57,
 'exposed': 58,
 'it': 59,
 'more': 60,
 'than': 61,
 '30': 62,
 'ago': 63,
 'researchers': 64,
 'reported': 65,
 'fiber': 66,
 'crocidolite': 67,
 'unusually': 68,
 'resilient': 69,
 'enter

In [12]:
tag2id

{'NNP': 0,
 ',': 1,
 'CD': 2,
 'NNS': 3,
 'JJ': 4,
 'MD': 5,
 'VB': 6,
 'DT': 7,
 'NN': 8,
 'IN': 9,
 '.': 10,
 'VBZ': 11,
 'VBG': 12,
 'CC': 13,
 'VBD': 14,
 'VBN': 15,
 'RB': 16,
 'TO': 17,
 'PRP': 18,
 'RBR': 19,
 'WDT': 20,
 'VBP': 21,
 'RP': 22,
 'PRP$': 23,
 'JJS': 24,
 'POS': 25,
 '``': 26,
 'EX': 27,
 "''": 28,
 'WP': 29,
 ':': 30,
 'JJR': 31,
 'WRB': 32,
 '$': 33,
 'NNPS': 34,
 'WP$': 35,
 '-LRB-': 36,
 '-RRB-': 37,
 'PDT': 38,
 'RBS': 39,
 'FW': 40,
 'UH': 41,
 'SYM': 42,
 'LS': 43,
 '#': 44}

In [43]:
max_len = 50
X_indices = [
    [vocab2id[palavra] for palavra, _ in sentenca]
    for sentenca in sentencas_treino
]
X = pad_sequences(sequences=X_indices, maxlen=max_len, padding="post", value=len(vocab)-1)

y_indices = [
    [tag2id[tag] for _, tag in sentenca]
    for sentenca in sentencas_treino
]
y = pad_sequences(sequences=y_indices, maxlen=max_len, padding="post", value=tag2id["."])
y = [to_categorical(seq, num_classes=len(tags)) for seq in y]

#### Separação em conjunto de treino e teste

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

### Definição de arquiteturas

#### Rede Neural

#### LSTM

##### Modelo

In [16]:
input_uni = Input(shape=(max_len,)) # Camada de Input
modelo_uni = Embedding(input_dim=len(vocab), output_dim=100, input_length=max_len)(input_uni) # Camada de Word embedding com dimensão maior
modelo_uni = Dropout(0.2)(modelo_uni) # Camada de Dropout com taxa menor
modelo_uni = LSTM(units=64, return_sequences=True, recurrent_dropout=0.2)(modelo_uni) # Camada de LSTM unidirecional com mais unidades
modelo_uni = Dropout(0.3)(modelo_uni) # Camada de Dropout adicional
modelo_uni = LSTM(units=32, return_sequences=True, recurrent_dropout=0.1)(modelo_uni) # Segunda camada LSTM com menos unidades
modelo_uni = Dense(128, activation='relu')(modelo_uni) # Camada densa intermediária
modelo_uni = Dropout(0.2)(modelo_uni) # Dropout após camada densa
out_uni = TimeDistributed(Dense(len(tags), activation="softmax"))(modelo_uni)  # Camada de softmax output

modelo_uni = Model(input_uni, out_uni) # Modelo completo

modelo_uni.summary()

c:\programming\PLN\pos-tagger-architechtures\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 50, 100)        │     4,734,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 50, 64)         │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50, 32)         │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50, 128)        │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 50, 45)         │         5,805 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,799,085 (18.31 MB)

 Trainable params: 4,799,085 (18.31 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
modelo_uni.compile(
    optimizer="adam", 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)

##### Treino

In [18]:
history = modelo_uni.fit(
    X_train, 
    np.array(y_train),
    batch_size=32, 
    epochs=3, 
    validation_split=0.2, 
    verbose=1
)

Epoch 1/3
985/985 ━━━━━━━━━━━━━━━━━━━━ 71s 67ms/step - accuracy: 0.6749 - loss: 1.2631 - val_accuracy: 0.9620 - val_loss: 0.1419
Epoch 2/3
985/985 ━━━━━━━━━━━━━━━━━━━━ 61s 62ms/step - accuracy: 0.9602 - loss: 0.1458 - val_accuracy: 0.9734 - val_loss: 0.0902
Epoch 3/3
985/985 ━━━━━━━━━━━━━━━━━━━━ 63s 64ms/step - accuracy: 0.9765 - loss: 0.0828 - val_accuracy: 0.9753 - val_loss: 0.0814


##### Salvar Modelo

In [19]:
import pickle

# Salvar o modelo treinado
with open('lstm_uni_model_1.pkl', 'wb') as f:
    pickle.dump(modelo_uni, f)

print("Modelo salvo com sucesso em 'lstm_uni_model_1.pkl'")

Modelo salvo com sucesso em 'lstm_uni_model_1.pkl'


##### Teste demonstrativo com sentença aleatória

In [ ]:
def predict_tags_lstm(texto, vocab2id, vocab, tags, modelo_lstm, max_len):
    # Tokenização
    tokens = regex.findall(r"\w+|[^\w\s]", texto.lower(), regex.UNICODE)

    # Tokens para índices
    INDICE_PAD = len(vocab) - 1
    ids_tokens = [vocab2id.get(token, INDICE_PAD) for token in tokens]

    # Padding
    sequencia_preenchida = pad_sequences(maxlen=max_len, sequences=[ids_tokens], padding="post", value=INDICE_PAD)

    # Predição
    predicao = modelo_lstm.predict(np.array([sequencia_preenchida[0]]))
    indices_tags = np.argmax(predicao, axis=-1)

    # Resultado
    resultado_tags = []
    for i, (indice_token, indice_tag) in enumerate(zip(sequencia_preenchida[0], indices_tags[0])):
        if i < len(ids_tokens):  # Só até o comprimento real da sentença
            palavra = vocab[indice_token]
            tag = tags[indice_tag]
            resultado_tags.append((palavra, tag))
    return resultado_tags

In [41]:
texto = "I am testing my model, finally it works!"
predict_tags_lstm(texto, vocab2id, vocab, tags, modelo_uni, max_len)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


[('i', 'PRP'),
 ('am', 'VBP'),
 ('testing', 'VBG'),
 ('my', 'PRP$'),
 ('model', 'NN'),
 (',', ','),
 ('finally', 'RB'),
 ('it', 'PRP'),
 ('works', 'VBZ'),
 ('!', '.')]

#### Bidirectional LSTM

##### Modelo

In [23]:
input_bi = Input(shape=(max_len,)) # Camada de Input
modelo_bi = Embedding(input_dim=len(vocab), output_dim=50, input_length=max_len)(input_bi) # Camada de Word embedding
modelo_bi = Dropout(0.3)(modelo_bi) # Camada de Dropout
modelo_bi = Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1))(modelo_bi) # Camada de Bidirectional LSTM
out_bi = TimeDistributed(Dense(len(tags), activation="softmax"))(modelo_bi)  # Camada de softmax output

modelo_bi = Model(input_bi, out_bi) # Modelo completo

modelo_bi.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 50, 50)         │     2,367,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 50, 100)        │        40,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 50, 45)         │         4,545 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,412,145 (9.20 MB)

 Trainable params: 2,412,145 (9.20 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
modelo_bi.compile(
    optimizer="adam", 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)

##### Treino

In [25]:
history = modelo_bi.fit(X_train, 
    np.array(y_train), 
    batch_size=16, 
    epochs=3, 
    validation_split=0.2, 
    verbose=1
)

Epoch 1/3
1969/1969 ━━━━━━━━━━━━━━━━━━━━ 85s 40ms/step - accuracy: 0.8053 - loss: 0.7968 - val_accuracy: 0.9785 - val_loss: 0.0713
Epoch 2/3
1969/1969 ━━━━━━━━━━━━━━━━━━━━ 76s 39ms/step - accuracy: 0.9829 - loss: 0.0579 - val_accuracy: 0.9827 - val_loss: 0.0540
Epoch 3/3
1969/1969 ━━━━━━━━━━━━━━━━━━━━ 77s 39ms/step - accuracy: 0.9879 - loss: 0.0375 - val_accuracy: 0.9832 - val_loss: 0.0521


##### Salvar Modelo

In [26]:
import pickle

# Salvar o modelo treinado
with open('lstm_bi_model_1.pkl', 'wb') as f:
    pickle.dump(modelo_bi, f)

print("Modelo salvo com sucesso em 'lstm_bi_model_1.pkl'")

Modelo salvo com sucesso em 'lstm_bi_model_1.pkl'


##### Teste demonstrativo com sentença aleatória

In [42]:
texto = "I am testing my model, finally it works!"
predict_tags_lstm(texto, vocab2id, vocab, tags, modelo_bi, max_len)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


[('i', 'PRP'),
 ('am', 'VBP'),
 ('testing', 'VBG'),
 ('my', 'PRP$'),
 ('model', 'NN'),
 (',', ','),
 ('finally', 'RB'),
 ('it', 'PRP'),
 ('works', 'VBZ'),
 ('!', '.')]